NAME:RAKSHITHA M

REG NO:2022510016


---



# **Sentence piece tokenization**

---



# **About the dataset**
Source:Extracted from various Tamil Wikipedia articles.

Content:Contains diverse topics (e.g., culture, science, history).



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# **import libraries**

In [ ]:
from pathlib import Path
import sentencepiece as spm
import pandas as pd

# **load data**

In [ ]:

gz_file = '/content/drive/My Drive/filtered_data.csv.gz'

lang_data = pd.read_csv(gz_file, compression='gzip', encoding='utf-8')

lang_data = lang_data.replace({'\ufffd': ''}, regex=True)

print(lang_data.head())


   Unnamed: 0     id                                        url  \
0           0  48482  https://ta.wikipedia.org/wiki?curid=48482   
1           1  48485  https://ta.wikipedia.org/wiki?curid=48485   
2           2  48486  https://ta.wikipedia.org/wiki?curid=48486   
3           3  48492  https://ta.wikipedia.org/wiki?curid=48492   
4           4  48493  https://ta.wikipedia.org/wiki?curid=48493   

                title                                               text  
0        தென் துருவம்  தென் துருவம் தென் முனை தென் துருவம் என்பது புவ...  
1    ஆர்க்டிக் வட்டம்  ஆர்க்டிக் வட்டம் ஆர்க்டிக் வட்டம் என்பது ஐந்து...  
2      நாஞ்சில் நாடன்  நாஞ்சில் நாடன் நாஞ்சில் நாடன் பிறப்பு திசம்பர்...  
3            டிக்கோயா  டிக்கோயா டிக்கோயா இலங்கையின் மத்திய மாகாணத்தின...  
4  நள்ளிரவுச் சூரியன்  நள்ளிரவுச் சூரியன் நள்ளிரவுச் சூரியன் அல்லது த...  


In [ ]:
lang_data.head()

,Unnamed: 0,id,url,title,text
0,0,48482,https://ta.wikipedia.org/wiki?curid=48482,தென் துருவம்,தென் துருவம் தென் முனை தென் துருவம் என்பது புவ...
1,1,48485,https://ta.wikipedia.org/wiki?curid=48485,ஆர்க்டிக் வட்டம்,ஆர்க்டிக் வட்டம் ஆர்க்டிக் வட்டம் என்பது ஐந்து...
2,2,48486,https://ta.wikipedia.org/wiki?curid=48486,நாஞ்சில் நாடன்,நாஞ்சில் நாடன் நாஞ்சில் நாடன் பிறப்பு திசம்பர்...
3,3,48492,https://ta.wikipedia.org/wiki?curid=48492,டிக்கோயா,டிக்கோயா டிக்கோயா இலங்கையின் மத்திய மாகாணத்தின...
4,4,48493,https://ta.wikipedia.org/wiki?curid=48493,நள்ளிரவுச் சூரியன்,நள்ளிரவுச் சூரியன் நள்ளிரவுச் சூரியன் அல்லது த...


# **Preprocessing**

In [ ]:
lang_data.drop('Unnamed: 0', axis=1, inplace=True)
print(" Removed 'Unnamed: 0' column.")


 Removed 'Unnamed: 0' column.


In [ ]:
lang_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133412 entries, 0 to 133411
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      133412 non-null  int64 
 1   url     133412 non-null  object
 2   title   133412 non-null  object
 3   text    133412 non-null  object
dtypes: int64(1), object(3)
memory usage: 4.1+ MB


In [ ]:
lang_data.shape

(133412, 4)

In [ ]:
OUTPUT_DIR = Path('/content/working')
TEXTS_DIR = OUTPUT_DIR/'texts'
TOK_DIR = OUTPUT_DIR/'tokenizer'

TOK_DIR.mkdir(parents=True, exist_ok=True)
TEXTS_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:

print(f" Duplicates: {lang_data.duplicated().sum()}")

lang_data.drop_duplicates(inplace=True)

print(f" Missing values:\n{lang_data.isnull().sum()}")

lang_data.dropna(inplace=True)
print(" Removed duplicates and missing values.")

 Duplicates: 0
 Missing values:
id       0
url      0
title    0
text     0
dtype: int64
 Removed duplicates and missing values.


In [ ]:
lang_data.shape

(133412, 4)

## **Saving dataset rows as text files**

In [ ]:
for t in lang_data.itertuples():
    file_name = Path(TEXTS_DIR/f'text_{t.Index}.txt')
    file_name.touch()
    with file_name.open('w') as f:
        f.write(t.text)

In [ ]:
len([t for t in TEXTS_DIR.iterdir()]), lang_data.shape[0]

(133412, 133412)

## **Displaying Generated File Names**

In [ ]:
files = ','.join([str(t) for t in TEXTS_DIR.iterdir()])
files[:100]

'/content/working/texts/text_90856.txt,/content/working/texts/text_48726.txt,/content/working/texts/t'

## **Training SentencePiece Tokenizer with Different Vocabulary Sizes**

In [ ]:
import sentencepiece as spm
for v in 8000, 16000, 20000, 30000:
    api_str = f"""--input={files} --vocab_size={v} --model_type=unigram --character_coverage=0.9995 --model_prefix={str(TOK_DIR)}/tok_{v}_size --max_sentence_length=20000"""
    print("Training with vocab set as:", v)
    spm.SentencePieceTrainer.train(api_str)

Training with vocab set as: 8000
Training with vocab set as: 16000
Training with vocab set as: 20000
Training with vocab set as: 30000


This code trains a SentencePiece tokenizer with different vocabulary sizes (8000, 16000, 20000, 30000).

Iteration: It loops over the four vocabulary sizes.

API String: Constructs the training parameters, including:

--input={files} → Uses the previously created text files as input.

--vocab_size={v} → Sets the vocabulary size dynamically for each iteration.

--model_type=unigram → Specifies the unigram language model.

--character_coverage=0.9995 → Covers 99.95% of the characters for accuracy.

--model_prefix={str(TOK_DIR)}/tok_{v}_size → Sets the output model prefix path.

--max_sentence_length=20000 → Defines the maximum sentence length for processing.

Training:

spm.SentencePieceTrainer.train(api_str) → Trains the tokenizer with the specified parameters.

Output:

Prints the current vocabulary size being used for training.

## **Listing All Files in a Directory Recursively**



In [ ]:
import os
for dirname, _, filenames in os.walk('/content'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Streaming output truncated to the last 5000 lines.
/content/working/texts/text_99795.txt
/content/working/texts/text_79455.txt
/content/working/texts/text_102033.txt
/content/working/texts/text_63848.txt
/content/working/texts/text_5578.txt
/content/working/texts/text_128021.txt
/content/working/texts/text_9534.txt
/content/working/texts/text_116445.txt
/content/working/texts/text_28547.txt
/content/working/texts/text_84413.txt
/content/working/texts/text_9273.txt
/content/working/texts/text_22349.txt
/content/working/texts/text_90194.txt
/content/working/texts/text_716.txt
/content/working/texts/text_68053.txt
/content/working/texts/text_14947.txt
/content/working/texts/text_26107.txt
/content/working/texts/text_3459.txt
/content/working/texts/text_85289.txt
/content/working/texts/text_72382.txt
/content/working/texts/text_57193.txt
/content/working/texts/text_80015.txt
/content/working/texts/text_84856.txt
/content/working/texts/text_17351.txt
/content/working/texts/text_46258.txt
/c

## **files saved:vocab and model of all sizes(8000,16000,20000,30000)**

## **List of Sentences for Tokenization**

In [ ]:
sentences = [

    'அவர் தினமும் யோகா செய்வார்.',

    'நேற்று மழை பெய்ததால் நான் வீட்டில் இருந்தேன்.',

    'ஒரு பூனை மூக்கு அடைப்பதற்கு இரண்டு இடைவெளிகள்.',

    'சமூக வலைதளங்கள் நவீன தகவல் பரிமாற்றத்துக்கு உதவுகின்றன.',
]


In [ ]:
import sentencepiece as spm
from pathlib import Path
from IPython.core.display import display, HTML
from string import Template

sp = spm.SentencePieceProcessor()

TOK_PATH = '/content/working/tokenizer'



## **Tokenizing and Displaying Results in HTML Table**

In [ ]:
from IPython.core.display import display, HTML
from string import Template

MODEL_PATHS = sorted(MODEL_PATHS, key=lambda p: int(p.stem.split('_')[1]))

td_text = Template('<td>$text</td>')
tr_text = Template('<tr><td>$name</td> $li_elem</tr>')

def tokenize_and_display_results(text):
    model_texts = []

    for model in MODEL_PATHS:

        sp.Load(str(model))

        tok = sp.EncodeAsPieces(text)

        word_html = ''.join([td_text.substitute(text=word) for word in tok])
        list_html = tr_text.substitute(name=model.stem, li_elem=word_html)
        model_texts.append(list_html)

    return display(HTML('<table>' + ''.join(model_texts) + '</table>'))


In [ ]:
tokenize_and_display_results(sentences[0])

In [ ]:
tokenize_and_display_results(sentences[1])

8000:

More fragmented tokens, less efficient.

Example: ▁பெய் + த → Split into two tokens.

16000 and 20000:

Slightly larger subwords, but still fragmented.

Example: ▁பெய் + த → Not fully merged.

30000:

Most compact representation.

Example: ▁பெய்த → Merged into a single token.

In [ ]:
tokenize_and_display_results(sentences[2])

8000:

More fragmented tokens, less efficient.

Example: ▁பூ + னை → Split into two tokens.

16000, 20000, 30000:

More compact and consistent tokenization.

Example: ▁பூனை → Merged into a single token.

Minimal difference between 20000 and 30000.

In [ ]:
tokenize_and_display_results(sentences[3])

8000:

More fragmented tokens, less efficient.

Example: ▁வலை + தள + ங்கள்.

16000:

Slightly larger subword units.

Example: ▁உதவ + ுகின்றன.

20000:

More compact tokens, better efficiency.

Example: ▁வலைதள + ங்கள்.

30000:

Most compact representation, minimal fragmentation.

Best for low-resource languages.



---

RESULT:
8000: More fragmented, less efficient.

16000 & 20000: Moderate efficiency, slight fragmentation.

30000: Most compact and efficient, ideal for Tamil NLP.

Best choice: 30000 for better tokenization and model performance.

## **Why Sentence piece tokenization instead of normal tokenization?:comparison**

In [ ]:
import sentencepiece as spm
import re

def standard_tokenizer(text):
    tokens = re.findall(r'\w+|[^\w\s]', text)
    return tokens


sp = spm.SentencePieceProcessor()
model_path = '/content/working/tokenizer/tok_30000_size.model'
sp.Load(model_path)

def sentencepiece_tokenizer(text):
    return sp.EncodeAsPieces(text)

for sentence in sentences:
    std_tokens = standard_tokenizer(sentence)
    sp_tokens = sentencepiece_tokenizer(sentence)

    print(f"\n Original: {sentence}")
    print(f" Standard Tokenization: {std_tokens}")
    print(f" SentencePiece Tokenization: {sp_tokens}")



 Original: அவர் தினமும் யோகா செய்வார்.
 Standard Tokenization: ['அவர', '்', 'த', 'ி', 'னம', 'ு', 'ம', '்', 'ய', 'ோ', 'க', 'ா', 'ச', 'ெ', 'ய', '்', 'வ', 'ா', 'ர', '்', '.']
 SentencePiece Tokenization: ['▁அவர்', '▁தினமும்', '▁யோகா', '▁செய்வார்', '.']

 Original: நேற்று மழை பெய்ததால் நான் வீட்டில் இருந்தேன்.
 Standard Tokenization: ['ந', 'ே', 'ற', '்', 'ற', 'ு', 'மழ', 'ை', 'ப', 'ெ', 'ய', '்', 'தத', 'ா', 'ல', '்', 'ந', 'ா', 'ன', '்', 'வ', 'ீ', 'ட', '்', 'ட', 'ி', 'ல', '்', 'இர', 'ு', 'ந', '்', 'த', 'ே', 'ன', '்', '.']
 SentencePiece Tokenization: ['▁நே', 'ற்று', '▁மழை', '▁பெய்த', 'தால்', '▁நான்', '▁வீட்டில்', '▁இருந்தே', 'ன்', '.']

 Original: ஒரு பூனை மூக்கு அடைப்பதற்கு இரண்டு இடைவெளிகள்.
 Standard Tokenization: ['ஒர', 'ு', 'ப', 'ூ', 'ன', 'ை', 'ம', 'ூ', 'க', '்', 'க', 'ு', 'அட', 'ை', 'ப', '்', 'பதற', '்', 'க', 'ு', 'இரண', '்', 'ட', 'ு', 'இட', 'ை', 'வ', 'ெ', 'ள', 'ி', 'கள', '்', '.']
 SentencePiece Tokenization: ['▁ஒரு', '▁பூனை', '▁மூக்கு', '▁அடை', 'ப்பதற்கு', '▁இரண்டு', '▁இடைவெளி', 'கள்



*  Standard Tokenization: Splits words into individual characters or morphemes.
*  SentencePiece: Retains subword units for meaningful chunks.

Example:
அவர் →

Standard: ['அவர', '்']

SentencePiece: ['▁அவர்']




*   Standard Tokenization:Generates more tokens, resulting in longer sequences.
*  SentencePiece:Produces fewer tokens, making it more efficient.


Example:

நேற்று மழை பெய்ததால் நான் வீட்டில் இருந்தேன்.

Standard: 36 tokens

SentencePiece: 9 tokens



---


# **Conclusion: Why SentencePiece Tokenization?**
Subword Tokenization:
Handles rare and OOV words efficiently.
Creates meaningful subword units.

Language Agnostic:
Works for Tamil and other low-resource languages.
No need for pre-segmentation.

Compact Representation:
Fewer tokens → Better efficiency.
Improves model performance.

Flexibility:
Supports multiple model types.
Easy to customize vocabulary size.